Notes
    
        
- Heaviside step function
- def step_fun(x,a,b,c): return a * (np.sign(x-b) + c)
- p0=[np.max(y), np.median(x), np.min(y)]


In [148]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import expit
from scipy.optimize import curve_fit
# from mplcursors import cursor
from dataclasses import dataclass
print("done")

done


In [149]:
def step_sim(step_size = 8.0, SNR = 0.5, total_time_point=400):
    np.random.seed(20181118)
    
    steps = np.piecewise(np.arange(0, total_time_point), 
                            [
                                np.arange(0, total_time_point) < 99, 

                                np.logical_and(np.arange(0, total_time_point) >= 99, np.arange(0, total_time_point) < 199), 

                                np.logical_and(np.arange(0, total_time_point) >= 199, np.arange(0, total_time_point) < 249), 
                                
                                np.logical_and(np.arange(0, total_time_point) >= 249, np.arange(0, total_time_point) < 349), 
                                
                                np.logical_and(np.arange(0, total_time_point) >= 349, np.arange(0, total_time_point) < 499), 

                                np.arange(0, total_time_point) >= 499
                            ], 
                            [
                                lambda x: 600, 
                                lambda x: 600 - ((x+1)%100), 
                                lambda x: 500 - ((x+1)%200), 
                                lambda x: 400 - ((x+1)%300),
                                lambda x: 300 - ((x+1)%400),
                                lambda x: 200 - ((x+1)%500),
                            ])
    x = np.arange(0, total_time_point)
    noise_STD = step_size / SNR
    noise = np.random.normal(scale=noise_STD, size=len(x))
    # y = true_fct + np.random.standard_t(1, size=len(x))
    y = steps + noise
    return x, y, steps

def step_sim_v2(step_size = 8.0, SNR = 0.5, total_time_point=400):
    np.random.seed(20181118)
    
    steps = np.piecewise(np.arange(0, total_time_point), 
                            [
                                np.arange(0, total_time_point) < 99, 

                                np.logical_and(np.arange(0, total_time_point) >= 99, np.arange(0, total_time_point) < 199), 

                                np.logical_and(np.arange(0, total_time_point) >= 199, np.arange(0, total_time_point) < 249), 
                                
                                np.logical_and(np.arange(0, total_time_point) >= 249, np.arange(0, total_time_point) < 349), 
                                
                                np.logical_and(np.arange(0, total_time_point) >= 349, np.arange(0, total_time_point) < 499), 

                                np.arange(0, total_time_point) >= 499
                            ], 
                            [
                                lambda x: 600, 
                                lambda x: 500, 
                                lambda x: 400 , 
                                lambda x: 300 ,
                                lambda x: 200 ,
                                lambda x: 100 ,
                            ])
    x = np.arange(0, total_time_point)
    noise_STD = step_size / SNR
    noise = np.random.normal(scale=noise_STD, size=len(x))
    # y = true_fct + np.random.standard_t(1, size=len(x))
    y = steps + noise
    return x, y, steps
print("done")

done


In [152]:

def moving_average(a, n=2, pad=False) :
    if pad:
        a = np.pad(a, (n-1,0))
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

@dataclass(repr=True)
class StepInfo:
    L:float
    x0:float
    k:float
    b:float
    location:float
    x:np.ndarray=None
    y:np.ndarray=None
    sqrt_error:np.ndarray=None

@dataclass(repr=True)
class FitInfo:
    fit_params: list
    location:float = 0
    x:np.ndarray = None
    y:np.ndarray = None

# @dataclass(repr=True)
# class StepInfo:
#     a:float
#     b:float
#     c:float
#     location:float
#     x:np.ndarray=None
#     y:np.ndarray=None
#     sqrt_error:np.ndarray=None


def _step_fun_logistic(x, L ,x0, k, b):
    # Logistic function
    # f(X, L, x0, k, b)
    # https://en.wikipedia.org/wiki/Logistic_function
    # use for curve fitting
    # https://stackoverflow.com/questions/55725139/fit-sigmoid-function-s-shape-curve-to-data-using-python
    y = L / (1 + np.exp(-k*(x-x0))) + b
        amplitude*(1. - 1./(1. + exp((x-center)/max(tiny, sigma))))
    return y

def _step_fun_heaviside(x, a, b, c): 
    return a * (np.sign(x-b) + c)



def fit_step(x, y, fun, p0):
    popt, pcov, infodict, mesg, ier = curve_fit(fun, x, y,p0=p0, full_output=True, maxfev = 1000000)
    return FitInfo(fit_params=popt)
    
def get_step_location_size(x, y):
    p0=[np.max(y), np.median(x),1, np.min(y)]
#     p0=[np.max(y), np.median(x), np.min(y)]
    popt, pcov, infodict, mesg, ier = curve_fit(_step_fun_logistic, x, y,p0=p0, full_output=True, maxfev = 1000000)
    L, x0, k, b = popt
    
#     uniques, indices, counts = np.unique(np.round(fit_y), return_counts=True, return_index=True)
    
    sqrt_error = np.sqrt(infodict['fvec']**2)
    location = x[np.where(sqrt_error==np.min(sqrt_error))[0][0]]
    info = StepInfo(L, x0, k, b, location, x, y, sqrt_error)
    
    ################
    y_fit = _step_fun_logistic(x, L, x0, k, b)

    uniques, indices, counts = np.unique(y_fit, return_counts=True, return_index=True)
    print(f"uniques:{uniques}, indices:{indices}, counts:{counts}")
#     print(y_fit)
    _indices =np.sort(indices)

    new_y = np.zeros(shape=y.shape)
    window_information = []
    for index, location in enumerate(_indices):
        if index == 0:
            new_y[0:location+1] = np.mean(y_data[0:location+1])
            window_loc = (x[0], x[location])
            window_information.append({'mean':np.mean(y[0:location+1]), 'window_loc':window_loc, 'window_size': abs(window_loc[1]-window_loc[0]) })
        else:
            pre_s = _indices[index-1]
            cur_s = _indices[index]
#             print(f"pre_s:{pre_s},cur_s:{cur_s}, len(x):{len(x)}, len(y):{len(y)}" )
            new_y[pre_s +1:cur_s+1] = np.mean(y[pre_s +1:cur_s+1])
            window_loc = (x[pre_s +1], x[cur_s])
            window_information.append({'mean':np.mean(y_data[pre_s +1:cur_s+1]), 'window_loc':window_loc, 'window_size': abs(window_loc[1]-window_loc[0])  })

        if index == len(indices)-1:
            new_y[location +1:] = np.mean(y[location +1:])
            window_loc = (x[location], x[-1]) if location == len(x)-1 else (x[location +1], x[-1])
            window_information.append({'mean':np.mean(y_data[location +1:]), 'window_loc':window_loc, 'window_size': abs(window_loc[1]-window_loc[0]) })


    y_std = np.std(y)
    selected_win = None
    max_hight = 0
    for index, win in enumerate(window_information[:-1]):
#         if (index+1) != (len(window_information)):
        window_hight = abs(win['mean'] - window_information[index+1]['mean'])
        if window_hight > max_hight:
            selected_win =  win
           
    window = selected_win
    ################
#     print(f"loc -> {window['window_loc'][0]}, {window['window_loc'][1]} : {window['window_loc']}")
    step = StepInfo(L, x0, k, b, window['window_loc'][-1], window['window_loc'][0], window['window_loc'][1])
    return step

def step_generator(x:np.ndarray, y:np.ndarray, steps:list, window_size_limit:int = 40):
    info = get_step_location_size(x, y)
    if info is None:
        return
    steps.append(info)
    location_index = np.where(x == info.location)[0][0]
    
    left_x = x[:location_index]
    left_y = y[:location_index]
    if len(left_y) > window_size_limit:
        step_generator(left_x, left_y, steps)
        
    right_x = x[location_index+1:]
    right_y = y[location_index+1:]
    if len(right_y)>window_size_limit:
        step_generator(right_x, right_y, steps)


def get_y_from_step_locations(steps, y):
    # Generate the new steped data using the step locations and the orignal y data
    std_y = np.std(y)
    new_y = np.zeros(shape=y.shape)
    for index, s in enumerate(steps):
        if index == 0:
            new_y[0:s+1] = np.mean(y[0:s+1])
        else:
            pre_s = steps[index-1]
            cur_s = steps[index]
            new_y[pre_s +1:cur_s+1] = np.mean(y[pre_s +1:cur_s+1])
    
        if index == len(steps)-1:
            new_y[s +1:] = np.mean(y[s +1:])
    return new_y

def get_step_hights(locations, y):
    window_hights = []
    steps = locations
#     window_height = np.std(y)
    
    first_step = steps.pop(0)
    if first_step == 0:
        first_step = steps.pop(0)
    
    first_win = np.mean(y[0: first_step+1])
    
#     if(np.std(win)>= window_height):
#         windows.append(first_step)
        
    while len(steps):
        next_step = steps.pop(0)
        next_win = np.mean(y[first_step+1:next_step+1])
        window_hights.append(abs(first_win - next_win))
#         if(abs(first_win - next_win)>= window_height):
#             windows.append(first_step)
        first_step = next_step
        first_win = next_win
    
    return window_hights

def filter_steps(locations, y, step_width=40):
    windows = []
    steps = locations
    window_height = np.std(y)
    
    first_step = steps.pop(0)
    if first_step == 0:
        first_step = steps.pop(0)
    
    first_win = np.mean(y[0: first_step+1])
    
#     if(np.std(win)>= window_height):
#         windows.append(first_step)
        
    while len(steps):
        next_step = steps.pop(0)
        next_win = np.mean(y[first_step+1:next_step+1])
        
        if(abs(first_win - next_win)>= window_height):
            windows.append(first_step)
        first_step = next_step
        first_win = next_win
    
    return windows
    
    
#     for index, s in enumerate(steps):
#         if index == 0:
#             new_y[0:s.location+1] = np.mean(y[0:s.location+1])
#         else:
#             pre_s = steps[index-1]
#             cur_s = steps[index]
#             new_y[pre_s.location +1:cur_s.location+1] = np.mean(y[pre_s.location +1:cur_s.location+1])
    
#         if index == len(steps)-1:
#             new_y[s.location +1:] = np.mean(y[s.location +1:])
#     return new_y
print("done")

done


In [151]:
# x_data, y_data , steps_org = step_sim_v2(total_time_point=1000, max_signal=2000)
x_data, y_data , steps_org = step_sim_v2(total_time_point=600)

In [171]:
from scipy.signal import find_peaks


In [ ]:
find_peaks()